In [16]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity , linear_kernel
import numpy as np
import pandas as pd 
import numpy as np
import re
import os
import argparse
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential,Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import SimpleRNN , Dropout ,LSTM,Conv1D, GlobalMaxPooling1D, MaxPooling1D , SpatialDropout1D,Activation,Dense, Dropout, Activation, Flatten, Input, concatenate , GRU
from keras.layers.embeddings import Embedding
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
#import xgboost as xgb

In [17]:
stop_words = stopwords.words('arabic')

In [18]:
data = pd.read_csv(r"C:\Users\Mahmoud\Desktop\cloud\full_dataset.tsv", delimiter = '\t' , quoting = 3)

In [19]:
X = data.iloc[: , 0].values
y = data.iloc[: , -1].values
print(X)
print(y)

['شيخ انا اسفه ف السؤال ده بس في ثوابت ف الدين بتضيع مني هو بجد كان في ايه ف القران وفي معزه اكلتها وحكمها لسه موجود لحد دلوقتي ف حد الزنا !!!!!!'
 'شيخ انا بصلي وانا قاعده انا كويسه صحتي حلوه اقدر اني اصلي وانا واقفه بس من 3 سنين جالي الوسواس القهري ف موض ع الصلاه بقيت ممكن اعيد الصلاه بالوضوء 5 او 6 مرات او اكتر ونلقتش حل عير اني اصلي وانا قاعده دلوقتي هل حرام ؟!'
 'لو سمحت لو بابا بيفطر رمضان عشان مريض بس المشكله انه مش بيطلع كفارة ّ....ممكن احنا نطلعها من وراه وتسقط عنه!!!وكمان لو عليه ديون من زمان لحد...حاجات بسيطة 20جنيه لحد وحاجات ف الرينج دا بس هو مطنشها ينفع نرجعها لاصحابها من وراه وتسقط عنه!!!!'
 ...
 'وليه بتفهم كلام ربنا غلط عايز تدرس المسيحيه بجد من غير اغلاط وتفهمعا هات الكتاب المقدس مع التفسير وانت هتعرف يعني ايه تكون مسيحي هتحس بسلام هتلاقي كلام ربنا كله محبه لابناءه هتلاقي العدل والخير هنشوف اد ايه بيحبك وعمل علشانك ايه ممكن تاخد كلامي بسخرية ممكن تقول انا قريته بس لو قريته كنت هتبقى مسيحي'
 '-' '-']
['هو حديث ضعيف، وعموما لو معزة كلت المصحف كله فالقرآن ثابت محفوظ. لا 

In [20]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)


def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text


def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text


def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        u"\u2066"
        u"\u2069"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', str(data))

def to_lowercase(text):
    return text.lower()

def remove_eng(text):
    return re.sub(r'[A-Za-z]','',str(text))
    
import string
def remove_punctuation(text):
    """Remove punctuation from list of tokenized words"""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)


def replace_numbers(text):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    return re.sub(r'\d+', '', text)


def remove_whitespaces(text):
    return text.strip()


def remove_stopwords(words, stop_words):

    return [word for word in words if word not in stop_words]

from nltk.stem.isri import ISRIStemmer
def lemmatize_words(words):
    """Lemmatize words in text"""

    lemmatizer = ISRIStemmer()
    return ' '.join([lemmatizer.stem(word) for word in words])

def text2words(text):
  return word_tokenize(text)

def normalize_text(text):
    text = remove_emojis(text)
    text = remove_eng(text)
    text = remove_punctuation(text)
    text = remove_punctuations(text)
    text = normalize_arabic(text)
    text = remove_diacritics(text)
    text = remove_repeating_char(text)
    text = to_lowercase(text)
    text = replace_numbers(text)
    words = text2words(text)
    words = remove_stopwords(words, stop_words)
    words = lemmatize_words(words)
    return ''.join(words)

def normalize_corpus(corpus):
  return [normalize_text(t) for t in corpus]

In [21]:
X = normalize_corpus(X)
#y = normalize_corpus(y)
print(X)
#print(y)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [22]:
vectorizer = TfidfVectorizer(ngram_range=(1,3), tokenizer=text2words,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1)
X = vectorizer.fit_transform(X)

In [24]:
while True:
    
    question = [input('Please enter a question: \n')]
    nor_input = normalize_corpus(question)
    if (len(nor_input[0]) != 0):
        nor_input = vectorizer.transform(nor_input)

        rank = linear_kernel(X , nor_input)
        #print(rank)
        sim_scores = list(enumerate(rank))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[0:1]
        courses_indices = [i[0] for i in sim_scores]
        for i in courses_indices:
            print('chat : ',data['Answer'].iloc[i])
    else:
        break

Please enter a question: 
هل السرقه حرام
chat :  نعم
Please enter a question: 
ما حكم الدين فى الزنااا
chat :  الحمد لله كفر بالله وصد عن سبيله
Please enter a question: 
ما الزنااا
chat :  الإنسان لا يبحث عن وزن الإثم بقدر بحثه عن التوبة والاعتذار لله سبحانه.
Please enter a question: 
هل الزنا حرام
chat :  مشوّهين فطرياً دي كلمة لا أحب استعمالها وأنا في سياق الطب النفسي. ربما منتكسين نفسياً أقرب في الوصف.
Please enter a question: 
ما حكم الدين فى الزانى
chat :  الحمد لله. الحرمان من وعد الله بالجنة، وحصول الإثم في قلبه، واسوداد جزء من قلبه، هذه أكبر عقوبة! الزاني خرج ممن وعد الله بدخول الجنة، فهو عاص لله، واقع في ما هو من أكبر الكبائر، فإما أن يتوب توبة تمحو هذه الخطيئة العظيمة، ويعود إلى دائرة الموعودين بالجنة. وإما أن يبقى مذلولا في مشيئة الله كسائر أهل الكبائر!
Please enter a question: 
كلمنى عن الجنه والنار وازاى ارضى ربنا
chat :  تفعل الفرائض وتجتنب المحرمات وتحسن أخلاقك، وإذا أذنبت فتب. وتحب الخير للمسلمين ويسلم صدرك لهم.
Please enter a question: 
هل انت اجتماعى
chat :  إلى حد ما